In [1]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv

In [2]:
field = galois.GF(7)
#definisco una matrice k x (n-k)
k = 7 #le righe
n_k = 7 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [3]:
max_column = 5
#number of iteration for Montecarlo simulatiom
num_iterations = 1000

In [4]:
#FOR IMPLEMENTS CF 4.4
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

#initial matrix x matrix with s or s^(-1)
def make_matrix_xS(matrix, field_order):
    #start = time.perf_counter() this is for total time
    matrix_S = field(np.zeros((k, k), dtype = int))
    for i, row in enumerate(matrix):
        #print("Type row: ", type(row)) IS GALOIS (CONFIRMED)
        s = np.sum(row)
        s_prime = np.sum(row ** (field_order-2))
        if s != 0:
            matrix_S[i,i] = np.reciprocal(s)
        elif s_prime != 0:
            matrix_S[i,i] = s_prime
        else:
            #end = time.perf_counter() this is for total time
            #duration = (end - start) * 1000 this is for total time
            print("esco")
            return []#, duration this is for total time
    #end = time.perf_counter() this is for total time
    #duration = (end - start) * 1000 this is for total time
    matrix = matrix_S @ matrix
    return matrix#, duration this is for total time

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix, error_counter):  
    if len(matrix) == 0:
        #no error_counter += 1 because isn t an error of algoritm
        return [], 0, 0, [], 0, 0, error_counter
    start = time.perf_counter()#this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts = np.zeros((matrix.shape[0], field.order + 1), dtype = int)
    n_colonne = matrix.shape[1]  
    #start = time.perf_counter()
    
    #for row in matrix:
    for i, row in enumerate(matrix):
        #bp.bincout count frequency of element in row
        vector_of_counts[i] = np.bincount(row, minlength = field.order + 1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration = (end - start) * 1000
    #vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        #print(vector_of_counts)
        print("Duplicated rows all columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(n_colonne, -1, -1)])[::-1]
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration = (end - start) * 1000    
    
    start = time.perf_counter() #this is for make file csv dati_differences_upd2 and only count/ord
    vector_of_counts1 = np.zeros((matrix.shape[0], max_column), dtype = int)
    for i, row in enumerate(matrix):
        #BY REMOVING THIS FILTERING (BELOW) WHICH WOULD ADD OVERHEAD AS I HAVE TO SCROLL THROUGH THE ENTIRE VECTOR REGARDLESS
        #I WILL SAVE IN TERMS OF TIME
        #valid_values = row[row < max_column]
        vector_of_counts1[i] = np.bincount(row, minlength = max_column)[:max_column]
    #print(vector_of_counts1)
    end = time.perf_counter() #these 2 for file only count/ord
    duration1 = (end - start) * 1000
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        #print(vector_of_counts1)
        print("Duplicated rows less columns",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    """for raw in matrix[sorted_index1]:
        print(np.sort(raw))"""
    #print()
    #end = time.perf_counter() #these 2 for make file csv dati_differences_upd2
    #duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, 0, matrix[sorted_index1], duration1, 0, error_counter
    else:
        print("If negativo")
        error_counter += 1
        print(error_counter)
        return [], duration, 0, [], duration1, 0, error_counter

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    #print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    """for raw in sorted_matrix:
        print(raw)
    print()"""
    return sorted_matrix, num_swap_columns


In [7]:
def check_non_zero_row(row):
    #return true if all elements != 0
    return np.all(row)

#make scaling matrix for every row with all elements != 0 and A x D added to a vector
def scaled_matrices(matrix):
    scaled_matrices = []
    scaling_matrix = field(np.zeros((k, n_k), dtype=int))
    for i, row in enumerate(matrix):
        if check_non_zero_row(row):
            for j in range(n_k):
                scaling_matrix[j,j] = np.reciprocal(matrix[i,j])
            print(f"Matrice per scalare\n", scaling_matrix)
            print("Matrix x scalata")
            print(matrix @ scaling_matrix)
            scaled_matrices.append(matrix @ scaling_matrix)

    return scaled_matrices

#aggiusta sta cazzo di cosa vedi come deve funzionare considerando che ho una lista di matrici e mi devo portare appresso
#i tempi e quant'altro
def apply_cf_4_4(scaled_matrices):
    all_matrices = []
    error_counter = 0
    num_swap_col = 0
    if len(scaled_matrices) != 0:
        for matrix in scaled_matrices:
            row_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row(matrix, error_counter)
            if not len(row_sort) == 0:
                scaled_matrix_sorted, num_swap_col = sort_by_column(row_sort, range(k), 0)
                print(scaled_matrix_sorted)
                all_matrices.append(scaled_matrix_sorted)
            else:
                #a questo punto provo a farla fallire poi vedo come gestirla
                return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, all_matrices
        return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, all_matrices
    
#1 METODO
#vorrei provare a una riga (la prima) da ogni matrice, crearmi una nuova matrice con queste righe, fare il conto delle 
#frequenze e vedere se ci sono righe uguali, se ci sono prendo tutte le seconde righe e faccio lo stesso, altrrimenti ok

#2 METODO
#dall'applicazione che fai in order_row del conteggio delle frequenze tornati quello della prima riga per cui posso fare il 
#confronto con quelle. Se ci sono uguaglianze anche qui prendi le seconde e cosi via.

#IN GENERALE FAI DELLE SIMULAZIONI PER VEDERE MEDIAMENTE QUANTE VOLTE CAPITA CHE IL VETTORE DELLE FREQUENZE DELLE PRIME RIGHE
#SIA UGUALE TRA MATRICI DIVERSE, COSI DA CERCARE UN LIMITE ACCETTABILE.
def cf5(all_matrices):
    smallest_matrix = []
    smallest_matrix = np.zeros((len(all_matrices), max_column), dtype = int)
    for i, matrix in enumerate(all_matrices):
        smallest_matrix[i] = np.bincount(matrix[0,:], minlength = max_column)[:max_column]

    print("This is frequence of every first line", smallest_matrix) 

    smallest_index = np.lexsort([smallest_matrix[:, i] for i in range(smallest_matrix.shape[1]-1, -1, -1)])[::-1]
    return all_matrices[smallest_index[0]]


In [9]:
A = field(np.random.randint(0, field.order, size=(k, n_k))) 
while(has_duplicate_rows(A)):
    A = field(np.random.randint(0, field.order, size=(k, n_k)))
print(f"Questa è quella di partenza\n", A)
matrici_scalate = scaled_matrices(A)
esito, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter, all_matrices = apply_cf_4_4(matrici_scalate)
smallest = cf5(all_matrices)
print(f"Smallest matrix --> CF5 = \n", smallest)

Questa è quella di partenza
 [[2 2 3 5 2 4 3]
 [1 3 0 2 6 4 4]
 [3 4 5 5 2 6 3]
 [3 1 2 6 3 4 5]
 [3 1 0 3 5 5 2]
 [4 6 4 5 6 3 5]
 [0 4 0 1 5 4 5]]
Matrice per scalare
 [[4 0 0 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 5 0 0 0 0]
 [0 0 0 3 0 0 0]
 [0 0 0 0 4 0 0]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 0 5]]
Matrix x scalata
[[1 1 1 1 1 1 1]
 [4 5 0 6 3 1 6]
 [5 2 4 1 1 5 1]
 [5 4 3 4 5 1 4]
 [5 4 0 2 6 3 3]
 [2 3 6 1 3 6 4]
 [0 2 0 3 6 1 4]]
Matrice per scalare
 [[5 0 0 0 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 3 0 0 0 0]
 [0 0 0 3 0 0 0]
 [0 0 0 0 4 0 0]
 [0 0 0 0 0 6 0]
 [0 0 0 0 0 0 5]]
Matrix x scalata
[[3 4 2 1 1 3 1]
 [5 6 0 6 3 3 6]
 [1 1 1 1 1 1 1]
 [1 2 6 4 5 3 4]
 [1 2 0 2 6 2 3]
 [6 5 5 1 3 4 4]
 [0 1 0 3 6 3 4]]
Matrice per scalare
 [[5 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 6 0 0 0]
 [0 0 0 0 5 0 0]
 [0 0 0 0 0 2 0]
 [0 0 0 0 0 0 3]]
Matrix x scalata
[[3 2 5 2 3 1 2]
 [5 3 0 5 2 1 5]
 [1 4 6 2 3 5 2]
 [1 1 1 1 1 1 1]
 [1 1 0 4 4 3 6]
 [6 6 2 2 2 6 1]
 [0 4 0 6 4 1 1]]
Matrice per scala